In [ ]:
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from selenium import webdriver
import time, re

In [ ]:
browser = webdriver.Chrome()
HOST = "https://www.ptt.cc"
headers = {
    'cookie': "over18=1"
}
browser.get("https://www.ptt.cc/bbs/Beauty/index.html")
soup = BeautifulSoup(browser.page_source,'lxml')    

In [ ]:
btn_list = soup.select("#action-bar-container > div > div.btn-group.btn-group-paging > a")
btn_list

In [ ]:
btn_list[1]
last_page = btn_list[1]['href']
last_page
last_page.split("index")
last_page.split("index")[1]
last_page.split("index")[1].split(".html")
page_str = last_page.split("index")[1].split(".html")[0]
page_str
page_int = int(page_str)
page_int

In [ ]:
PAGE_TO_CRAWL = 40
URL_TEMPLATE = "/bbs/Beauty/index{}.html"
for page in range(page_int, page_int - PAGE_TO_CRAWL, -1):
    print("[DEBUG] Crawling page %s"%(page))
    URL = HOST + URL_TEMPLATE.format(page)
    print("[DEBUG] Current page is %s"%(URL))

In [ ]:
def crawl_list(url):
    # input: <String> list page url
    # output: <List> list of article urls
    headers = {
        'cookie': "over18=1"
    }
    res = requests.get(url, headers=headers)
    #print("[DEBUG] res page is %s"%(res.text))
    soup = BeautifulSoup(res.text, 'lxml')
    a_tags = soup.select("#main-container > div.r-list-container.action-bar-margin.bbs-screen > div > div.title > a")
    article_urls = [HOST + link['href'] for link in a_tags]
    return article_urls

In [ ]:
crawl_list("https://www.ptt.cc/bbs/Beauty/index2118.html")

In [ ]:
def crawl_page(url):
    # input: <String> article page url
    # output: <Dictionary> article object
    print("[DEBUG] crawl_page at %s"%(url))
    article_dict = {}
    
    headers = {
        'cookie': "over18=1"
    }
    res = requests.get(url, headers=headers)
    #print("[DEBUG] res page is %s"%(res.text))
    soup = BeautifulSoup(res.text, 'lxml')
    meta_list = soup.select("#main-content > div > span.article-meta-value") # [kohanchen (kohanchen), Gossiping...]
    article_dict['author'] = meta_list[0].text
    article_dict['board'] = meta_list[1].text
    article_dict['title'] = meta_list[2].text
    article_dict['dt'] = meta_list[3].text
    
    article_dict['ip'] = soup.select_one("#main-content").text.split("發信站: 批踢踢實業坊(ptt.cc), 來自: ")[1].split("\n※ 文章網址:")[0].strip()
    
    # 刪除不需要的elemets....
    for meta in meta_list:
        meta.extract()
    for meta in soup.select("#main-content > div > span.article-meta-tag"):
        meta.extract()
    for push in soup.select("div.push"):
        push.extract() 
    for push in soup.select("span.f2"):
        push.extract()

    article_dict['content'] = soup.select_one("#main-content").text.strip()
    
    return article_dict

In [ ]:
import logging # https://docs.python.org/2/library/logging.html
logging.basicConfig(filename='crawler.log',level=logging.DEBUG)
logger = logging.getLogger("crawler")
logger.debug('This message should go to the log file')


PAGE_TO_CRAWL = 3
URL_TEMPLATE = "/bbs/Beauty/index{}.html"
for page in range(page_int, page_int - PAGE_TO_CRAWL, -1):
    print("[DEBUG] Crawling page %s"%(page))
    URL = HOST + URL_TEMPLATE.format(page)
    print("[DEBUG] Current page is %s"%(URL))
    for article in crawl_list(URL):
        try:
            print(article[37:42])
            res = requests.get(article)
            soup = BeautifulSoup(res.text,'lxml')
            photo = soup.select("#main-content > a")
            for count in range(1,len(photo)):
                print(photo[count]['href'])
                urlretrieve(photo[count]['href'], "%s_photo_%d.jpg" %(article[37:42],count))
        except Exception:
            logger.error("crawl_page failed, url is %s"%(article))
